In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [2]:
import sys
# sys.path.append('../')
sys.path.append('../')
sys.path.append('../model')
sys.path.append('../cost')
sys.path.append('../constraints')
sys.path.append('../utils')
import Reentry
from scipy.integrate import solve_ivp
from Scvx_tf_free import Scvx_tf_free
from Scaling import TrajectoryScaling

In [3]:
ix = 6
iu = 2
tf = 3.4
N = 99
delT = tf/N
max_iter = 10

In [4]:
Re = 6371*1000
ge = 9.806

scl_d = Re;
scl_v = np.sqrt(ge*Re);

x0  = 0.0/scl_d; 
y0  = 0.0/scl_d; 
h0  = 122e3/scl_d;
v0  = 7315.2/scl_v; 
gam0 = np.deg2rad(-1.5); 
thet0 = np.deg2rad(4); 

xi = np.array([x0,y0,h0,v0,gam0,thet0])
xf = np.array([100,100,100,10,0,0])
print(xi,xf)

[ 0.          0.          0.01914927  0.92550007 -0.02617994  0.06981317] [100 100 100  10   0   0]


In [5]:
x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf

u0 = np.zeros((N+1,iu))
u0[:,0] = 1.7 # setting not as zero is pretty critical
u0[:,1] = 0.0

In [6]:
myModel = Reentry.Reentry('Hello',ix,iu,'numeric_central')

# test continuous

In [7]:
x = np.expand_dims(np.array([1,2,3,4,5,6]),0)
u = np.expand_dims(np.array([100,20]),0)
print_np(x)

Type is <class 'numpy.ndarray'>
Shape is (1, 6)


In [8]:
A,B = myModel.diff_numeric(x,u,discrete=False)

In [9]:
A

array([[ 0.        ,  0.        ,  0.        ,  0.96017029,  0.        ,
         1.11764734],
       [ 0.        ,  0.        ,  0.        , -0.2794155 ,  0.        ,
         3.84068541],
       [ 0.        ,  0.        ,  0.        ,  5.        ,  4.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.06249988,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

# test discrete

In [10]:
import scipy.io
path = '/Users/keemtw/Documents/Code/Personal/reentryopt-master-purna/scp_tools/testing_foh/zbar_save.mat'
mat = scipy.io.loadmat(path)

In [11]:
zbar = np.array(mat['zbar_']).T

In [12]:
zbar,u0

(array([[ 0.00000000e+00,  0.00000000e+00,  1.91492701e-02,
          9.25500073e-01, -2.61799388e-02,  6.98131701e-02],
        [ 3.17074103e-02,  2.21719812e-03,  1.82325882e-02,
          9.25499106e-01, -3.15001350e-02,  6.98131701e-02],
        [ 6.34145717e-02,  4.43437883e-03,  1.71474346e-02,
          9.25494191e-01, -3.68087870e-02,  6.98131701e-02],
        [ 9.51218207e-02,  6.65156567e-03,  1.58928220e-02,
          9.25491428e-01, -4.21238420e-02,  6.98131701e-02],
        [ 1.26828867e-01,  8.86873832e-03,  1.44695551e-02,
          9.25472262e-01, -4.73891528e-02,  6.98131701e-02],
        [ 1.58533976e-01,  1.10857755e-02,  1.28826018e-02,
          9.25375843e-01, -5.24222585e-02,  6.98131701e-02],
        [ 1.90231991e-01,  1.33023167e-02,  1.11466742e-02,
          9.24975159e-01, -5.65393747e-02,  6.98131701e-02],
        [ 2.21896839e-01,  1.55165385e-02,  9.33623783e-03,
          9.22969215e-01, -5.58069233e-02,  6.98131701e-02],
        [ 2.53399154e-01,  1.771

In [13]:
print_np(zbar)
print_np(u0)

Type is <class 'numpy.ndarray'>
Shape is (100, 6)
Type is <class 'numpy.ndarray'>
Shape is (100, 2)


In [14]:
start = time.time()
A,Bm,Bp,s,z,x_prop = myModel.diff_discrete_foh(zbar[:N],u0,delT,tf)
end = time.time()
print(end - start)

0.06257987022399902


In [15]:
start = time.time()
# A_,Bm_,Bp_,s_,z_,x_prop_ = myModel.diff_discrete_foh_serial(zbar[:N],u0,delT,tf)
A_,Bm_,Bp_,s_,z_,x_prop_ = myModel.diff_discrete_foh_test2(zbar[:N],u0,delT,tf)
end = time.time()
print(end - start)

0.05095481872558594


In [16]:
for mat,mat_ in zip([A,Bm,Bp,s,z,x_prop],[A_,Bm_,Bp_,s_,z_,x_prop_]) :
    print(np.max(np.abs(mat - mat_)))

2.2597819153702403e-06
0.008152254109683385
0.0016758036026419051
0.004292108009445827
3.011916460959885e-07
9.45404825605789e-10


In [17]:
print_np(A_)

Type is <class 'numpy.ndarray'>
Shape is (99, 6, 6)


In [18]:
print_np(Bm_)

Type is <class 'numpy.ndarray'>
Shape is (99, 6, 2)
